## Основной код

In [8]:
import pandas as pd
import numpy as np
import keras
from tensorflow.keras import layers
from tensorflow.keras import models
import matplotlib.pyplot as plt

<h3>Блок функций</h3>

In [13]:
def data_preparation():
    train_data = pd.read_csv('train.csv', sep=';', dtype={'Source': np.object, 'Destination': np.object,  
                                                     'Protocol': np.object, 'Length': np.int64, 'Source Port': np.int64,
                                                     'Dest Port': np.int64, 'Delta time': np.float64, 'Error': np.object})
    train_data = train_data.fillna('delivered')

    categories = {}
    for key, value in enumerate(train_data['Error'].unique()):
        categories[value] = key + 1

    train_data['Dict_of_Error'] = train_data['Error'].map(categories)
    
    train_data = train_data.sample(frac=1).reset_index(drop=True)    #Функция перемештвания строк
    
    return train_data

In [7]:
def load_data_from_arrays(strings, labels, train_test_split=0.7):
    """Функция разделяет набор данный на тренировочную и тустовую выборки"""
    data_size = len(strings)
    test_size = int(data_size - round(data_size * train_test_split))
    print("Test size: {}".format(test_size))

    print("\nTraining set:")
    x_train = strings[test_size:]
    print("\t - x_train: {}".format(len(x_train)))
    y_train = labels[test_size:]
    print("\t - y_train: {}".format(len(y_train)))

    print("\nTesting set:")
    x_test = strings[:test_size]
    print("\t - x_test: {}".format(len(x_test)))
    y_test = labels[:test_size]
    print("\t - y_test: {}".format(len(y_test)))

    return x_train, y_train, x_test, y_test

<h3>Подготовка и обработка данных</h3>
<p><em>Надо понять, как признаки превратить в вектор и передать их в модель. 
    Возможно нужно сделать словарь, как для 'Error'</em></p>

In [14]:
train_db = data_preparation()
train_db.head()

,Source,Destination,Protocol,Length,Source Port,Dest Port,Delta time,Error,Dict_of_Error
0,10.0.17.129,10.0.4.66,TCP,2966,36384,445,0.000005,tcp_out_of_order,4
1,10.0.16.14,10.0.151.43,TCP,64,57511,8000,0.000118,delivered,1
2,10.0.4.66,10.0.17.129,TCP,70,445,36384,0.000001,delivered,1
3,10.0.17.129,10.0.4.66,TCP,2966,36384,445,0.000004,tcp_out_of_order,4
4,10.0.17.129,10.0.4.66,TCP,2966,36384,445,0.000003,tcp_out_of_order,4


In [ ]:

categories = train_db['Dict_of_Error']

X_train, y_train, X_test, y_test = load_data_from_arrays(description, categories)

In [ ]:



print('y_train shape:', y_train.shape)
print('y_test shape:', y_test.shape)

<h3>Создание модели</h3>
<p><em>Пока не работает</em></p>

In [ ]:
epochs = 10

model = models.Sequential()
model.add(layers.Dense(512, input_shape=(1000,)))
model.add(layers.Activation('relu'))
model.add(layers.Dropout(0.2))
model.add(layers.Dense(4))
model.add(layers.Activation('softmax'))

In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

print(model.summary())

In [ ]:
history = model.fit(X_train, y_train,
                    batch_size=32,
                    epochs=epochs,
                    verbose=1,
                    )
score = model.evaluate(X_test, y_test,
                       batch_size=32, verbose=1)

<h3>Тут будут графики</h3>

In [ ]:
# График точности модели
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# График оценки loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()